In [2]:
import pygame, random
from pygame.locals import *

SCREEN_WIDTH = 1000
SCREEN_HEIGHT = 600

GAME_SPEED = 10
GRAVITY = 40
SPEED = 1

GROUND_WIDTH  = SCREEN_WIDTH
GROUND_HEIGHT = SCREEN_HEIGHT - 30

End1 = SCREEN_WIDTH/2 + 100
End2 = -300

class Pik(pygame.sprite.Sprite):

    def __init__(self):
        pygame.sprite.Sprite.__init__(self)

        self.images = [pygame.image.load('Pikachu 1.png').convert_alpha(),
                       pygame.image.load('Pikachu 2.png').convert_alpha(),
                       pygame.image.load('Pikachu 3.png').convert_alpha(),
                       pygame.image.load('Pikachu 4.png').convert_alpha()]

        self.current_image = 0

        self.image = pygame.image.load('Pikachu 1.png').convert_alpha()
        self.mask  = pygame.mask.from_surface(self.image)

        self.rect    = self.image.get_rect()
        self.rect[0] = SCREEN_WIDTH / 2 - 200
        self.rect[1] = GROUND_HEIGHT - 250

    def update(self):
        self.current_image = (self.current_image + 1) % 4
        self.image = self.images[ self.current_image ]

        # Update height
        if self.rect[1] < GROUND_HEIGHT - 250: self.rect[1] += GRAVITY
        
    def jump(self): 
        if self.rect[1] >= GROUND_HEIGHT - 250: 
           self.rect[1] -= 200
           self.rect[0] += 100
            
    def go_on(self): 
        if self.rect[0] < 700: self.rect[0] += 30
    
    def go_back(self): 
        if self.rect[0] > 0: self.rect[0] -= 30

class Pok(pygame.sprite.Sprite):

    def __init__(self):
        pygame.sprite.Sprite.__init__(self)

        self.images = [pygame.image.load('Pigeotto.png').convert_alpha(),
                       pygame.image.load('Pigeotto 2.png').convert_alpha()]

        self.current_image = 0

        self.image = pygame.image.load('Pigeotto.png').convert_alpha()
        self.mask  = pygame.mask.from_surface(self.image)

        self.rect = self.image.get_rect()
        self.rect[0] = 0
        self.rect[1] = 0

    def update(self):
        self.current_image = (self.current_image + 1) % 2
        self.image = self.images[ self.current_image ]
    
    def pursuit(self, r0):
        if self.rect[0] < r0 - 30: self.rect[0] += 20
        else: self.rect[0] -= 10
        
    def hunt(self, r0, r1):
        if self.rect[1] <  r1 and self.rect[1] < GROUND_HEIGHT - 350: self.rect[1] += 30
        if self.rect[1] >= GROUND_HEIGHT - 350: self.rect[0] += 30
    
    def go_back(self):
        if self.rect[0] > 0: self.rect[0] -= 30
        if self.rect[1] > 0: self.rect[1] -= 30
            
    def give_up(self):
        if self.rect[1] < End1: self.rect[0] += 30
        if self.rect[1] > End2: self.rect[1] -= 10

class Ground(pygame.sprite.Sprite):
    def __init__(self, xpos):
        pygame.sprite.Sprite.__init__(self)

        self.image = pygame.image.load('layer-5.png').convert_alpha()
        self.image = pygame.transform.scale(self.image, (SCREEN_WIDTH, SCREEN_HEIGHT))
        self.mask = pygame.mask.from_surface(self.image)
        self.rect = self.image.get_rect()
        self.rect[0] = xpos
    
    def update(self): self.rect[0] -= GAME_SPEED
        
def is_off_screen(sprite): return sprite.rect[0] < -(sprite.rect[2])

pygame.init()
screen = pygame.display.set_mode((SCREEN_WIDTH, SCREEN_HEIGHT))

bird_group = pygame.sprite.Group()
bird = Pik()
bird_group.add(bird)

BACKGROUND = pygame.image.load('battleback6.png')
BACKGROUND = pygame.transform.scale(BACKGROUND, (SCREEN_WIDTH, SCREEN_HEIGHT))

bird_group2 = pygame.sprite.Group()
bird2 = Pok()
bird_group2.add(bird2)

# Ground:
ground_group = pygame.sprite.Group()
for i in range(2):
    ground = Ground(GROUND_WIDTH * i)
    ground_group.add(ground)
    
# Music:
pygame.mixer.music.load("Vicegrip of Pursuit.mp3")
pygame.mixer.music.play()
    
# Sounds:
atack = pygame.mixer.Sound("vulture-1.wav")

clock = pygame.time.Clock()    

c = 0
vidas = 1

while True:
    clock.tick(10)
    
    for event in pygame.event.get():
        if event.type == QUIT:
            pygame.quit()
            
    # movimentos do pikachu
    comandos = pygame.key.get_pressed()
    if comandos[pygame.K_SPACE]: bird.jump()
    if comandos[pygame.K_RIGHT]: bird.go_on()
    if comandos[pygame.K_LEFT]:  bird.go_back()    
    
    c += 1         # marcador de tempo
    
    # movimentos do pidgeotto:
    
    # 1 Ataque:
    if c <= 50:            bird2.pursuit(bird.rect[0])                 # pursuit
    if c >  40 and c < 45: atack.play()                                # som de ataque    
    if c >  50 and c < 60: bird2.hunt(bird.rect[0], bird.rect[1])      # hunt
    if c >= 60 and c < 80: bird2.go_back()                             # voltando
                                                                        
    # 2 Ataque:                                                         
    if c >= 80 and  c <= 130: bird2.pursuit(bird.rect[0])               # pursuit
    if c > 120 and  c <  125: atack.play()                              # som de ataque 
    if c > 130 and  c <  140: bird2.hunt(bird.rect[0], bird.rect[1])    # hunt
    if c >= 140 and c <  160: bird2.go_back()                           # voltando
                                                                        
    # 3 Ataque:                                                         
    if c >= 160 and c <= 200: bird2.pursuit(bird.rect[0])              # pursuit
    if c >  190 and c <  195: atack.play()                             # som de ataque 
    if c >  200 and c <  210: bird2.hunt(bird.rect[0], bird.rect[1])   # hunt
    if c >= 210 and c <  220: bird2.go_back()                          # voltando
                                                                        
    if c >= 225: bird2.give_up()                                       # desistindo
    
    screen.blit(BACKGROUND, (0, 0))
    
    if is_off_screen(ground_group.sprites()[0]):
        ground_group.remove(ground_group.sprites()[0])
        new_ground = Ground(GROUND_WIDTH - 10)
        ground_group.add(new_ground)
    
    ground_group.update()
    bird_group.update()
    bird_group2.update()
    
    ground_group.draw(screen)
    bird_group.draw(screen)
    bird_group2.draw(screen)
    
    pygame.display.update()
    
    if pygame.sprite.groupcollide(bird_group, bird_group2, 
                                  False, False, 
                                  pygame.sprite.collide_mask):
        vidas -= 1
        
    if c == 250:
        if vidas > 0: print("Wins!!")
        else: print("Try again losser!!")
        

error: video system not initialized